In [5]:
import csv
import pandas as pd
import scipy.spatial.distance as scidist


df = pd.read_csv('B2P_Rwanda_matchedIDs_final_2020-09-24_copy.csv', index_col=0)


In [6]:
df_project_coord = df[['project_code','lat','long']]
df_project_coord.head()

,project_code,lat,long
0,1014107,-2.420560,28.966200
1,1014106,-2.424860,28.957260
2,1007651,-1.720530,30.081240
3,1012493,-1.655950,30.078840
4,1014318,-1.870868,29.877686


In [7]:
df_population = pd.read_csv('population_rwa_2018-10-01.csv')

In [8]:
df_population.head()

,Lat,Lon,Population
0,-1.050417,30.463194,8.07593
1,-1.050417,30.463472,8.07593
2,-1.050694,30.463194,8.07593
3,-1.050694,30.464306,8.07593
4,-1.050972,30.445694,8.07593


In [9]:
# manual fix for R127 line # 534
df_project_coord.iloc[534]['project_code'] = 88888

In [10]:
# def euclidean_distance(x1,x2,y1,y2):

In [11]:
# df[df_project_coord['project_code']=='R127']

In [6]:
# a = df_population[:100]

In [7]:
# b = df_project_coord

In [12]:
# a.iloc[:,0:2]

In [13]:
# b.iloc[:,1:]

In [8]:
# distances = scidist.cdist(a.iloc[:,0:2],b.iloc[:,1:],'euclidean')
# distances

In [9]:
# closest = []
# for k,v in enumerate(distances):
#     # mini = min(v)
#     # pos = v.index(mini)
#     # print(min(v), v.argmin(axis=0))
#     closest.append(v.argmin(axis=0))

In [10]:
# distances[1].argmin(axis=0)

In [11]:
# df_closest = pd.DataFrame(closest)
# df_closest

In [12]:
# df_closest[0].value_counts()

150    100
Name: 0, dtype: int64

In [13]:
# df_closest.to_csv('closest_1.csv')

In [18]:
# df_closest = pd.DataFrame(closest)
# df_closest.index = pd.RangeIndex(100,100 + len(df_closest))

In [14]:
# df_closest

in a separate python file alled 'closest.py' we assign each of the 1.4 million coordinates in df_population, a project project_code.
below, we'll use the glob package to zip the closest1,.... closest15.csv and then merge this with the popuation table

In [15]:
import glob as glob

In [16]:
import glob as glob
closest_files = sorted(glob.glob('closest*.csv'))
closest_files

['closest01.csv',
 'closest02.csv',
 'closest03.csv',
 'closest04.csv',
 'closest05.csv',
 'closest06.csv',
 'closest07.csv',
 'closest08.csv',
 'closest09.csv',
 'closest10.csv',
 'closest11.csv',
 'closest12.csv',
 'closest13.csv',
 'closest14.csv',
 'closest15.csv']

In [17]:
closest_df =  pd.concat((pd.read_csv(file).assign(filename = file)for file in closest_files),ignore_index = True)

In [18]:
closest_df

,Unnamed: 0,0,filename
0,0,150,closest01.csv
1,1,150,closest01.csv
2,2,150,closest01.csv
3,3,150,closest01.csv
4,4,150,closest01.csv
...,...,...,...
1449192,1449192,736,closest15.csv
1449193,1449193,736,closest15.csv
1449194,1449194,736,closest15.csv
1449195,1449195,736,closest15.csv


In [19]:
closest_df = pd.merge(closest_df,df_population,left_index=True,right_index=True)
closest_df

,Unnamed: 0,0,filename,Lat,Lon,Population
0,0,150,closest01.csv,-1.050417,30.463194,8.07593
1,1,150,closest01.csv,-1.050417,30.463472,8.07593
2,2,150,closest01.csv,-1.050694,30.463194,8.07593
3,3,150,closest01.csv,-1.050694,30.464306,8.07593
4,4,150,closest01.csv,-1.050972,30.445694,8.07593
...,...,...,...,...,...,...
1449192,1449192,736,closest15.csv,-2.825694,29.539861,7.69858
1449193,1449193,736,closest15.csv,-2.825972,29.519861,7.69858
1449194,1449194,736,closest15.csv,-2.825972,29.524028,7.69858
1449195,1449195,736,closest15.csv,-2.826250,29.530417,7.69858


In [20]:
closest_df = closest_df[['0','Population']]
closest_df

,0,Population
0,150,8.07593
1,150,8.07593
2,150,8.07593
3,150,8.07593
4,150,8.07593
...,...,...
1449192,736,7.69858
1449193,736,7.69858
1449194,736,7.69858
1449195,736,7.69858


In [21]:
project_population_df = closest_df.groupby(['0'])['Population'].agg('sum').reset_index()
project_population_df

,0,Population
0,0,369.10581
1,1,383.07240
2,2,1176.66124
3,3,20025.22819
4,4,6882.17536
...,...,...
1391,1417,23613.22631
1392,1418,15874.60832
1393,1419,10601.81035
1394,1420,6494.84688


In [22]:
df_project_with_population = pd.merge(project_population_df,df_project_coord,left_index=True,right_index=True)
df_project_with_population

,0,Population,project_code,lat,long
0,0,369.10581,1014107,-2.420560,28.966200
1,1,383.07240,1014106,-2.424860,28.957260
2,2,1176.66124,1007651,-1.720530,30.081240
3,3,20025.22819,1012493,-1.655950,30.078840
4,4,6882.17536,1014318,-1.870868,29.877686
...,...,...,...,...,...
1391,1417,23613.22631,1013561,-2.293363,29.238419
1392,1418,15874.60832,1013562,-2.323533,29.254731
1393,1419,10601.81035,1013569,-2.301919,29.242738
1394,1420,6494.84688,1013565,-2.322038,29.247239


In [23]:
df_project_with_population.describe()

,0,Population,lat,long
count,1396.000000,1396.000000,1396.000000,1396.000000
mean,711.388968,8316.378697,-2.041703,29.583500
std,411.376563,11501.684384,0.489846,2.281788
min,0.000000,42.297520,-2.799865,-29.496450
25%,355.750000,2275.173030,-2.362840,29.434712
50%,709.500000,4760.182750,-2.075490,29.668372
75%,1067.250000,9818.560258,-1.733164,29.946835
max,1421.000000,146317.617160,2.794150,30.813100


In [39]:
df_project_with_population.Population.sum()

11609664.661539972

In [24]:
df_project_with_population.to_csv('projects_population.csv')